<a href="https://colab.research.google.com/github/eng-accelerator/ai-accelerator/blob/main/Day_8/autoclasses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install datasets evaluate transformers diffusers accelerate ftfy pyarrow --quiet

## Tokenizer

In [ ]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
raw_inputs = [
    "I am learning Operating Systems and it's so fun.",
    "I write terrible C++ code!",
]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt", max_length=15)
print(inputs)

{'input_ids': tensor([[ 101, 1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569,
         1012,  102],
        [ 101, 1045, 4339, 6659, 1039, 1009, 1009, 3642,  999,  102,    0,    0,
            0,    0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}


In [ ]:
# another way to do it - a closer look into the tokenizer
sequence = "I am learning Operating Systems and it's so fun."
tokens = tokenizer.tokenize(sequence)

print("--------TOKENIZED SENTENCE--------")
print(tokens)

print("\n\n--------TOKENS MAPPED TO THEIR IDS--------")
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

--------TOKENIZED SENTENCE--------
['i', 'am', 'learning', 'operating', 'systems', 'and', 'it', "'", 's', 'so', 'fun', '.']


--------TOKENS MAPPED TO THEIR IDS--------
[1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569, 1012]


**TAKE A PAUSE!**

**SPECIAL TOKENS!**

In [ ]:
# already tokenized input
decoded_string = tokenizer.decode([1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569, 1012])
print(decoded_string)

i am learning operating systems and it's so fun.


In [ ]:
# raw text
decoded_string = tokenizer.decode([101, 1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569,
         1012,  102])
print(decoded_string)

[CLS] i am learning operating systems and it's so fun. [SEP]


**Padding - ways to do it**

In [ ]:
# Will pad the sequences up to the maximum sequence length
model_inputs = tokenizer(sequences, padding="longest")

# Will pad the sequences up to the model max length
# (512 for BERT or DistilBERT)
model_inputs = tokenizer(sequences, padding="max_length")

# Will pad the sequences up to the specified max length
model_inputs = tokenizer(sequences, padding="max_length", max_length=8)

## Models

In [ ]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape) # bs, seq_len, dim

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)

print("--------SHAPE OF LOGIT TENSOR--------")
print(outputs.logits.shape)

print("\n\n--------LOGIT TENSOR--------")
print(outputs.logits)


predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = torch.argmax(predictions, dim=1)
print("\n\n--------PREDICTIONS--------")
print(predictions)

--------SHAPE OF LOGIT TENSOR--------
torch.Size([2, 2])


--------LOGIT TENSOR--------
tensor([[-4.3357,  4.6875],
        [ 4.6717, -3.7884]], grad_fn=<AddmmBackward0>)


--------PREDICTIONS--------
tensor([1, 0])


In [ ]:
model.config.id2label

{0: 'NEGATIVE', 1: 'POSITIVE'}